# Embeddings Retrieval with Claude

This notebook provides a step-by-step guide for using the Embedding search tool with Claude. We will:

1. Set up the environment and imports
2. Load documents into a vector store (either local or remote)
3. Build a search tool to query the Brave search engine
4. Test the search tool  
5. Create a Claude client with access to the tool 
6. Compare Claude's responses with and without access to the tool

## Imports and Configuration 

First we'll import libraries and load environment variables. This includes setting up logging so we can monitor the process.

In [1]:
import os
import sys
import dotenv
import anthropic

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), os.pardir)))

import claude_retriever

# Load environment variables
dotenv.load_dotenv()

True

In [2]:
# Import and configure logging 
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

# Create a handler to log to stdout
handler = logging.StreamHandler(sys.stdout)
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

## Store your data

The first step is setting up your datastore. Here, we will make use of the [Kaggle Amazon Products 2020 Dataset](https://www.kaggle.com/datasets/promptcloud/amazon-product-dataset-2020). It contains 10000 products from Amazon, including their product title, description, price, category tags, etc. For the purposes of this notebook, we've pre-processed the data to concatenate the title, description and category tags into a single "document" field and saved it locally as a JSONL with one line for each product.

We now need to transform this raw text dataset into an embedding dataset. In this notebook we will opt for the simplest possible way to do this locally:

1. We will use the [sentence-transformers](https://www.sbert.net/index.html) library, which allows us to use a lightweight model to embed our text data using only a CPU if that is all we have available.
2. We will save the text/embedding pairs on disk as a JSONL file that can be loaded in memory on the fly.

### Note: Dense vs Sparse Hybrid Vector Stores
There are also two main options for the type of vectors stored:

 - Dense vectors: Traditional dense embeddings like sentence transformers. These capture semantic meaning well but ignore exact keyword matches.
 - Sparse hybrid vectors: A combination of dense vectors plus extra dimensions for exact keyword matching. This allows combining semantic search with keyword search.

We show how to use dense vectors by default, but include code to use hybrid vectors. The hybrid approach requires changing the vector store, embedder, and search tool.

## Local retrieval

In [3]:
# Initialize the vector store
from claude_retriever.searcher.vectorstores.local import LocalVectorStore, LocalHybridVectorStore
from claude_retriever.searcher.embedders.local import LocalHybridEmbedder
from claude_retriever.utils import embed_and_upload 
from claude_retriever.constants import DEFAULT_EMBEDDER, DEFAULT_SPARSE_EMBEDDER

vector_store = LocalVectorStore("data/amazon-products-embeddings.jsonl")
########## Delete the line above and uncomment the lines below to use hybrid vector stores. ##########
# vector_store = LocalHybridVectorStore("data/amazon-products-hybrid-embeddings.jsonl")

if len(vector_store.embeddings) == 0:
    # Fill the vector store from local text files
    logger.info("Vector store is empty. Filling it from local text files.")
    batch_size = 128
    input_file = "data/amazon-products.jsonl"

    embed_and_upload(input_file, vector_store, batch_size=batch_size)
    ########## Delete the line above and uncomment the lines below to use hybrid vector stores. ##########
    # embedder = LocalHybridEmbedder(DEFAULT_EMBEDDER, DEFAULT_SPARSE_EMBEDDER)
    # embed_and_upload(input_file=input_file, vectorstore=vector_store, embedder=embedder, batch_size=batch_size)

2023-08-29 13:24:04,448 - claude_retriever.searcher.vectorstores.local - INFO - Creating empty datastore at data/amazon-products-embeddings.jsonl
2023-08-29 13:24:04,449 - claude_retriever.searcher.vectorstores.local - INFO - Loading datastore from data/amazon-products-embeddings.jsonl
2023-08-29 13:24:04,449 - root - INFO - Vector store is empty. Filling it from local text files.
2023-08-29 13:24:04,449 - claude_retriever.utils - INFO - Using default embedder: sentence-transformers/all-mpnet-base-v2
2023-08-29 13:24:04,450 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Embedding and uploading documents: 100%|██████████| 10239/10239 [14:45<00:00, 11.56it/s]


## Remote retrieval

Local methods like the above work quite well for small datasets, but for larger datasets you may want to consider using a cloud-based method to both create the embeddings and store the vector datastore. In this example, we create a [Pinecone](https://www.pinecone.io/) datastore.

In [ ]:
# One-off: setup index if it doesn't exist
import pinecone
from claude_retriever.searcher.vectorstores.pinecone import PineconeHybridVectorStore
from claude_retriever.searcher.embedders.local import LocalHybridEmbedder
from claude_retriever.utils import embed_and_upload
from claude_retriever.constants import DEFAULT_EMBEDDER, DEFAULT_SPARSE_EMBEDDER

PINECONE_API_KEY = os.environ["PINECONE_API_KEY"]
PINECONE_ENVIRONMENT = os.environ["PINECONE_ENVIRONMENT"]
PINECONE_DATABASE = os.environ["PINECONE_DATABASE"]

pinecone.init(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT)
if PINECONE_DATABASE not in pinecone.list_indexes():
  logger.info("No remote vectorstore found. Creating new index and filling it from local text files.")

  batch_size = 128
  input_file = "data/amazon-products.jsonl"

  pinecone.create_index(PINECONE_DATABASE, dimension=768, metric="cosine")
  vector_store = PineconeVectorStore(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT, index=PINECONE_DATABASE)
  embed_and_upload(input_file, vector_store, batch_size=batch_size)
  ########## Delete the three lines above and uncomment the lines below to use hybrid vector stores. ##########
  # pinecone.create_index(PINECONE_DATABASE, dimension=768, metric="dotproduct") # Pinecone only supports hybrid retrieval with dotproduct
  # vector_store = PineconeHybridVectorStore(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT, index=PINECONE_DATABASE)
  # embedder = LocalHybridEmbedder(DEFAULT_EMBEDDER, DEFAULT_SPARSE_EMBEDDER)
  # embed_and_upload(input_file=input_file, vectorstore=vector_store, embedder=embedder, batch_size=batch_size)
else:
  vector_store = PineconeVectorStore(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT, index=PINECONE_DATABASE)
  ########## Delete the line above and uncomment the line below to use hybrid vector stores. ##########
  # vector_store = PineconeHybridVectorStore(api_key=PINECONE_API_KEY, environment=PINECONE_ENVIRONMENT, index=PINECONE_DATABASE)

## Create a search tool
Using the vector store we just populated, let's create an EmbeddingSearchTool.

In [4]:
from claude_retriever.searcher.searchtools.embeddings import EmbeddingSearchTool

AMAZON_SEARCH_TOOL_DESCRIPTION = 'The search engine will search over the Amazon Product database, and return for each product its title, description, and a set of tags.'

amazon_search_tool = EmbeddingSearchTool(tool_description=AMAZON_SEARCH_TOOL_DESCRIPTION,
                                         vector_store = vector_store)

########## Delete the line above and uncomment the lines below if using hybrid vector stores. ##########
# embedder = LocalHybridEmbedder(DEFAULT_EMBEDDER, DEFAULT_SPARSE_EMBEDDER)
# amazon_search_tool = EmbeddingSearchTool(tool_description=AMAZON_SEARCH_TOOL_DESCRIPTION,
#                                          embedder=embedder,
#                                          vector_store = vector_store)

2023-08-29 13:49:21,024 - claude_retriever.searcher.searchtools.embeddings - INFO - Using default embedder: sentence-transformers/all-mpnet-base-v2
2023-08-29 13:49:21,025 - sentence_transformers.SentenceTransformer - INFO - Load pretrained SentenceTransformer: sentence-transformers/all-mpnet-base-v2


Let's test to see if the search tool works!

In [5]:
dinos = amazon_search_tool.search("fun kids dinosaur book", n_search_results_to_use=1)
print(dinos)


<search_results>
<item index="1">
<page_content>
Product Name: LeapFrog Dino's Delightful Day Alphabet Book, Green

About Product: Letters and words are woven into the story in alphabetical order with phonetic sounds to introduce ABCs to your little one through a charming tale | Flip through the 16 interactive pages to hear the story read aloud, or enjoy musical play by jamming to a melody with fun sounds and musical notes | Press the light-up button to hear letter names, letter sounds and words from the story | Number buttons along Dino's back introduce counting and recognizing numbers from one to ten | This complete story with beginning, middle and end exposes your child to early reading skills. 2AA batteries are included for demo purposes, replace new batteries for regular use. Product dimensions: 12.3" Wide x 12.5" Height x 2.7" Depth

Categories: Toys & Games | Learning & Education | Science Kits & Toys
</page_content>
</item>
</search_results>


## Use Claude with Retrieval
We can now simply pass this search tool to Claude to use in retrieval.

In [8]:
ANTHROPIC_SEARCH_MODEL = "claude-2"

client = claude_retriever.ClientWithRetrieval(api_key=os.environ['ANTHROPIC_API_KEY'], search_tool = amazon_search_tool)

query = "I want to get my daughter more interested in science. What kind of gifts should I get her?"
prompt = f'{anthropic.HUMAN_PROMPT} {query}{anthropic.AI_PROMPT}'

Here is the basic response to the query (with no access to the tool).

In [9]:
basic_response = client.completions.create(
    prompt=prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=ANTHROPIC_SEARCH_MODEL,
    max_tokens_to_sample=1000,
)
print('-'*50)
print('Basic response:')
print(prompt + basic_response.completion)
print('-'*50)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
2023-08-29 13:50:45,379 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
--------------------------------------------------
Basic response:


Human: I want to get my daughter more interested in science. What kind of gifts should I get her?

Assistant: Here are some science-themed gift ideas to help spark your daughter's interest:

- A kids telescope or microscope set - These allow hands-on exploration and discovery. Get one powerful enough to see planets/moon or magnify interesting specimens.

- Science kits - Kits for chemistry, biology, robotics, geology etc allow structured scientific discovery through experiments. Pick ag

Now we get the same completion, but give Claude the ability to use the tool when thinking about the response.

In [10]:
augmented_response = client.completion_with_retrieval(
    query=query,
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=3,
    max_tokens_to_sample=1000)

print('-'*50)
print('Augmented response:')
print(prompt + augmented_response)
print('-'*50)

2023-08-29 13:51:09,177 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
2023-08-29 13:51:09,184 - claude_retriever.client - INFO -  <thinking>
To gather relevant information to help the user find science gifts for their daughter, I should search for science kits, books, and toys that are designed for kids and focus on making science interesting and engaging. I'll search for top rated and popular science gifts for children in different age ranges to get a broad set of options.
</thinking>

<search_query>science gifts for kids
2023-08-29 13:51:09,191 - claude_retriever.client - INFO - Attempting search number 0.
2023-08-29 13:51:09,193 - claude_retriever.client - INFO - 
--------------------
Pausing stream because Claude has issued a query in <search_query> tags: <search_query>science gifts for kids</search_query>
--------------------
2023-08-29 13:51:09,194 - claude_retriever.client - INFO - Running search query against SearchTool: science gif


Often, you'll want finer-grained control around how exactly Claude uses the results. For this workflow we recommend "retrieve then complete".

In [11]:
relevant_search_results = client.retrieve(
    query=query,
    stop_sequences=[anthropic.HUMAN_PROMPT, 'END_OF_SEARCH'],
    model=ANTHROPIC_SEARCH_MODEL,
    n_search_results_to_use=3,
    max_searches_to_try=5,
    max_tokens_to_sample=1000)

print('-'*50)
print('Relevant results:')
print(relevant_search_results)
print('-'*50)

2023-08-29 13:52:05,279 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
2023-08-29 13:52:05,284 - claude_retriever.client - INFO -  <thinking>
To gather information to help answer this query, I should search for science kits, books, toys, and other products designed for kids that promote science education and engagement. The ideal results will cover a range of science subjects like biology, chemistry, physics, astronomy, etc. and be targeted for the appropriate age range. I'll start with some broad searches and then refine my queries if needed.
</thinking>

<search_query>science gifts for kids
2023-08-29 13:52:05,288 - claude_retriever.client - INFO - Attempting search number 0.
2023-08-29 13:52:05,289 - claude_retriever.client - INFO - 
--------------------
Pausing stream because Claude has issued a query in <search_query> tags: <search_query>science gifts for kids</search_query>
--------------------
2023-08-29 13:52:05,290 - claude_retrieve

Here we create a new prompt for answering the user's query using the retrieved search results.

In [12]:
qa_prompt = f'''{anthropic.HUMAN_PROMPT} You are a friendly product recommender. Here is a query issued by a user looking for product recommendations:

{query}

Here are a set of search results that might be helpful for answering the user's query:

{relevant_search_results}

Once again, here is the user's query:

<query>{query}</query>

Please write a response to the user that answers their query and provides them with helpful product recommendations. Feel free to use the search results above to help you write your response, or ignore them if they are not helpful.

At the end of your response, under "Products you might like:", list the top 3 product names from the search results that you think the user would most like.

Please ensure your results are in the following format:

<result>
Your response to the user's query.
</result>
<recommendations>
Products you might like:
1. Product name
2. Product name
3. Product name
</recommendations>{anthropic.AI_PROMPT}'''

response = client.completions.create(
    prompt=qa_prompt,
    stop_sequences=[anthropic.HUMAN_PROMPT],
    model=ANTHROPIC_SEARCH_MODEL,
    max_tokens_to_sample=1000,
)

print('-'*50)
print('Response:')
print(response.completion)
print('-'*50)

2023-08-29 13:52:53,855 - httpx - INFO - HTTP Request: POST https://api.anthropic.com/v1/complete "HTTP/1.1 200 OK"
--------------------------------------------------
Response:
 <result>

Based on your interest in getting your daughter more interested in science, here are some great gift ideas to consider:

For hands-on science fun, a science experiment kit is a great way to let her explore different scientific concepts. The Scientific Explorer My First Science Kit has age-appropriate experiments like growing crystals, making rainbows, and exploring colors. Kits like this introduce STEM principles in a creative way. 

Books and magazines that focus on science topics are another engaging option. The Newmark Learning science books cover various science topics at an early reading level. Getting a subscription to a kids science magazine full of experiments, facts, and more could provide ongoing science engagement.

Visiting museums, planetariums, zoos, aquariums, and other science-focused 